In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import yfinance as yf
from datetime import date

In [9]:
def load_data(ticker, start, end):
    data = yf.download(ticker, start, end)
    data.reset_index(inplace=True)
    return data

In [10]:
def create_sequences(data, sequence_length):
    sequences = []
    target = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length, 0]
        label = data[i+sequence_length, 0]
        sequences.append(seq)
        target.append(label)
    return np.array(sequences), np.array(target)

In [11]:
START = "2014-01-01"
TODAY = date.today().strftime("%Y-%m-%d")
selected_stock = 'AAPL'
data = load_data(selected_stock, START, TODAY)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

[*********************100%%**********************]  1 of 1 completed


In [12]:
sequence_length = 50
X, y = create_sequences(scaled_data, sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

In [14]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [15]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
62/62 [==============================] - 2s 21ms/step - loss: 0.0461 - val_loss: 0.0031
Epoch 2/50
62/62 [==============================] - 1s 17ms/step - loss: 0.0011 - val_loss: 7.7909e-04
Epoch 3/50
62/62 [==============================] - 1s 15ms/step - loss: 5.4999e-04 - val_loss: 5.3433e-04
Epoch 4/50
62/62 [==============================] - 1s 15ms/step - loss: 4.5112e-04 - val_loss: 4.7543e-04
Epoch 5/50
62/62 [==============================] - 1s 15ms/step - loss: 4.1948e-04 - val_loss: 4.8152e-04
Epoch 6/50
62/62 [==============================] - 1s 16ms/step - loss: 3.9568e-04 - val_loss: 4.8113e-04
Epoch 7/50
62/62 [==============================] - 1s 16ms/step - loss: 4.2091e-04 - val_loss: 5.2874e-04
Epoch 8/50
62/62 [==============================] - 1s 15ms/step - loss: 3.8633e-04 - val_loss: 4.2978e-04
Epoch 9/50
62/62 [==============================] - 1s 15ms/step - loss: 3.7459e-04 - val_loss: 4.2696e-04
Epoch 10/50
62/62 [==============================

In [16]:
model.save('stock_predict_model.h5')

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
